In [ ]:
from datetime import datetime

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
class EmailSummary(BaseModel):
    sender_name: str = Field(description="Sender's name")
    sender_email: str = Field(description="Sender's email address")
    receiver_name: str = Field(description="Receiver's name")
    receiver_email: str = Field(description="Receiver's email address")
    subject: str = Field(description="Title of the email")
    summary: str = Field(description="Summary of the email")
    date: datetime = Field(description="Meeting date and time mentioned on the email")


pydantic_parser: PydanticOutputParser[EmailSummary] = PydanticOutputParser(
    pydantic_object=EmailSummary
)

In [ ]:
print(pydantic_parser.get_format_instructions())

In [ ]:
template: str = """You are a helpful assistant.
Understand the email content in English if it's not an English.
Answer the following questions in KOREAN.
If dealing with date and time, convert it into datetime data with UTC timezone standard.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""


prompt = PromptTemplate.from_template(template=template).partial(
    format=pydantic_parser.get_format_instructions()
)
llm: ChatOllama = ChatOllama(model="llama3.2:3b")

chain = prompt | llm

In [ ]:
email_conversation: str = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
llm_with_structered = ChatOllama(model="llama3.2:3b").with_structured_output(schema=EmailSummary)
structured_answer = llm_with_structered.invoke(input=email_conversation)

print(structured_answer)